In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=eabd6555842a95b2e2cdc150f054819be098af73ed53a39e2e26e71a77448205
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


Here are the different Spark Modules.
 
Spark Core - RDD and Map Reduce APIs

Spark Data Frames and Spark SQL

Spark Structured Streaming

Spark MLLib (Data Frame based)

As engineers, we need not focus too much on Spark Core libraries to build Data Pipelines. We should focus on Spark Data Frames as well as Spark SQL

Spark Cluster Manager Types
Let us get an overview of different Spark Cluster Managers on which typically Spark Applications are deployed.

Here are the supported cluster manager types.

Local (used for development and unit testing).

Stand Alone

YARN

Mesos
Here are the popular distributions which use YARN to deploy Spark Applications.
Cloudera

AWS EMR

Google Dataproc

Hortonworks

MapR

Databricks uses Stand Alone for running or deploying Spark Job

As part of this section we will get an overview about Data Processing using Spark with Python.

    Pre-requisites and Objectives
    
    Starting Spark Context
    
    Overview of Spark read APIs
    
    Understand airlines data
    
    Inferring Schema
    
    Previewing airlines data
    
    Overview of Data Frame APIs
    
    Overview of Functions
    
    Overview of Spark Write APIs
    
    Reorganizing airlines data
   
    Previewing reorganized data
    
    Analyze and Understand Data
    
    Conclusion

et us say spark is of type SparkSession. There is an attribute as part of spark called as catalog and it is of type pyspark.sql.catalog.Catalog.

We can access catalog using spark.catalog.

We can permanently or temporarily create tables or views on top of data in a Data Frame.

Metadata such as table names, column names, data types etc for the permanent tables or views will be stored in Metastore. We can access the metadata using spark.catalog 
which is exposed as part of SparkSession object.

spark.catalog also provide us the details related to temporary views that are being created. Metadata of these temporary views will not be stored in Spark Metastore.

Permanent tables are typically created using databases in spark metastore. If not specified, the tables will be created in default database.

There are several methods that are part of spark.catalog. We will explore them in the later topics.

Following are some of the tasks that can be performed using spark.catalog object.
Check current database and switch to different databases.

Create permanent table in metastore.

Create or drop temporary views.

Register functions.

All the above tasks can be performed using SQL style commands passed to spark.sql.

In [2]:
from pyspark.sql import SparkSession

SparkSession is a class that is part of pyspark.sql package.It is a wrapper on top of Spark Context.

When Spark application is submitted using spark-submit or spark-shell or pyspark, a web service called as Spark Context will be started.

Spark Context maintains the context of all the jobs that are submitted until it is killed.

SparkSession is nothing but wrapper on top of Spark Context.

We need to first create SparkSession object with any name. But typically we use spark. Once it is created, several APIs will be exposed including read.

We need to at least set Application Name and also specify the execution mode in 
which Spark Context should run while creating SparkSession object.

We can use appName to specify name for the application and master to specify the execution mode.

Below is the sample code snippet which will start the Spark Session object for u

In [3]:
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/kaggle/working/warehouse"). \
    appName('Getting Started'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 09:09:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read the data from different sources.

    Files

    Databases

    Mainframes

    APIs

Processing the data

    Row Level Transformations

    Aggregations

    Sorting

    Ranking

    Joining multiple data sets

Write data to different targets.

    Files

    Databases

    Mainframes

    APIs

All APIs are exposed under spark.read
    
    text - to read single column data from text files as well as reading each of the whole text file as one record.
    
    csv- to read text files with delimiters. Default is a comma, but we can use other delimiters as well.
    
    json - to read data from JSON files
    
    orc - to read data from ORC files
    
    parquet - to read data from Parquet files.
We can also read data from other file formats by plugging in and by using spark.read.format

We can also pass options based on the file formats.
    inferSchema - to infer the data types of the columns based on the data.
    header - to use header to get the column names in case of text files.
    schema - to explicitly specify the schema.

We can get the help on APIs like spark.read.csv using help(spark.read.csv).
Reading delimited data from text files.

In [7]:
Trial_dataFrame = spark. \
    read. \
    csv(path='/kaggle/input/emr-dataset/Food_Establishment_Inspection_Data.csv',
        header=True,
       inferSchema=True)

In [8]:
Trial_dataFrame.show(2)

+----------------+------------------+---------------+--------------------+---------------+-------+--------+--------------+-------------+-----------+------------------------+--------------------+----------------+-----------------+--------------------------+--------------+---------------------+----------------+-----------+---------------------+-------------------+-----+
|            Name|Program Identifier|Inspection Date|         Description|        Address|   City|Zip Code|         Phone|    Longitude|   Latitude|Inspection Business Name|     Inspection Type|Inspection Score|Inspection Result|Inspection Closed Business|Violation Type|Violation Description|Violation Points|Business_ID|Inspection_Serial_Num|Violation_Record_ID|Grade|
+----------------+------------------+---------------+--------------------+---------------+-------+--------+--------------+-------------+-----------+------------------------+--------------------+----------------+-----------------+--------------------------+--

We can use options such as header and inferSchema to assign names and data types.
However inferSchema will end up going through the entire data to assign schema. 

We can use samplingRatio to process fraction of data and then infer the schema.

In case if the data in all the files have similar structure, we should be able to get the schema using one file and then apply it on others.

In our airlines data, schema is consistent across all the files and hence we should be able to get the schema by going through one file and apply on the entire dataset.

In [9]:
Trial_dataFrame.schema

StructType([StructField('Name', StringType(), True), StructField('Program Identifier', StringType(), True), StructField('Inspection Date', StringType(), True), StructField('Description', StringType(), True), StructField('Address', StringType(), True), StructField('City', StringType(), True), StructField('Zip Code', StringType(), True), StructField('Phone', StringType(), True), StructField('Longitude', DoubleType(), True), StructField('Latitude', DoubleType(), True), StructField('Inspection Business Name', StringType(), True), StructField('Inspection Type', StringType(), True), StructField('Inspection Score', IntegerType(), True), StructField('Inspection Result', StringType(), True), StructField('Inspection Closed Business', BooleanType(), True), StructField('Violation Type', StringType(), True), StructField('Violation Description', StringType(), True), StructField('Violation Points', IntegerType(), True), StructField('Business_ID', StringType(), True), StructField('Inspection_Serial_Nu

In [10]:
Trial_dataFrame.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Program Identifier: string (nullable = true)
 |-- Inspection Date: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Inspection Business Name: string (nullable = true)
 |-- Inspection Type: string (nullable = true)
 |-- Inspection Score: integer (nullable = true)
 |-- Inspection Result: string (nullable = true)
 |-- Inspection Closed Business: boolean (nullable = true)
 |-- Violation Type: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Violation Points: integer (nullable = true)
 |-- Business_ID: string (nullable = true)
 |-- Inspection_Serial_Num: string (nullable = true)
 |-- Violation_Record_ID: string (nullable = true)
 |-- Grade: integer (nullable =

In [11]:
yelpData = spark.read.json(path='/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')

In [12]:
yelpData.count()

6990280

In [15]:
yelpData.schema

StructType([StructField('business_id', StringType(), True), StructField('cool', LongType(), True), StructField('date', StringType(), True), StructField('funny', LongType(), True), StructField('review_id', StringType(), True), StructField('stars', DoubleType(), True), StructField('text', StringType(), True), StructField('useful', LongType(), True), StructField('user_id', StringType(), True)])

In [14]:
yelpData.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [18]:
yelpData.distinct().count()

6990280

In [17]:
Trial_dataFrame.distinct().count()

251455

In [17]:
Trial_dataFrame.distinct()

251455

In [19]:
attendants = [(1, "Smith", "Deo", 1086.0, "Somalia"),
             (2, "Kenry", "Jord", 1950.0, "Iran"),
             (3, "Kick", "Boxer", 7550.0, "Kingome"),
             (4, "Gill", "Nome", 1500.0, "New Wold")
            ]

In [20]:
type(attendants)

list

In [21]:
attendantsDF = spark.createDataFrame(attendants,schema='''attendant_id INT, f_name STRING,
                                     l_name STRING, salary FLOAT, Nationality STRING''')

In [39]:
from pyspark.sql.functions import *

In [22]:
attendantsDF.show()

+------------+------+------+------+-----------+
|attendant_id|f_name|l_name|salary|Nationality|
+------------+------+------+------+-----------+
|           1| Smith|   Deo|1086.0|    Somalia|
|           2| Kenry|  Jord|1950.0|       Iran|
|           3|  Kick| Boxer|7550.0|    Kingome|
|           4|  Gill|  Nome|1500.0|   New Wold|
+------------+------+------+------+-----------+



In [23]:
attendantsDF.select("f_name","Nationality").show()

+------+-----------+
|f_name|Nationality|
+------+-----------+
| Smith|    Somalia|
| Kenry|       Iran|
|  Kick|    Kingome|
|  Gill|   New Wold|
+------+-----------+



In [37]:
attendantsDF.withColumn('new_name',concat('Nationality',lit('_'),'l_name')).show()

+------------+------+------+------+-----------+-------------+
|attendant_id|f_name|l_name|salary|Nationality|     new_name|
+------------+------+------+------+-----------+-------------+
|           1| Smith|   Deo|1086.0|    Somalia|  Somalia_Deo|
|           2| Kenry|  Jord|1950.0|       Iran|    Iran_Jord|
|           3|  Kick| Boxer|7550.0|    Kingome|Kingome_Boxer|
|           4|  Gill|  Nome|1500.0|   New Wold|New Wold_Nome|
+------------+------+------+------+-----------+-------------+



In [38]:
attendantsDF.selectExpr("salary * 2 as new_salary","concat(f_name,l_name) as new_fname").show()

+----------+---------+
|new_salary|new_fname|
+----------+---------+
|    2172.0| SmithDeo|
|    3900.0|KenryJord|
|   15100.0|KickBoxer|
|    3000.0| GillNome|
+----------+---------+



In [24]:
attendantsDF.drop('Nationality').show()

+------------+------+------+------+
|attendant_id|f_name|l_name|salary|
+------------+------+------+------+
|           1| Smith|   Deo|1086.0|
|           2| Kenry|  Jord|1950.0|
|           3|  Kick| Boxer|7550.0|
|           4|  Gill|  Nome|1500.0|
+------------+------+------+------+



In [30]:
type(attendantsDF.drop('Nationality'))

pyspark.sql.dataframe.DataFrame

In [28]:
type(attendantsDF.drop('Nationality').collect()[0])

pyspark.sql.types.Row

In [41]:
yelpData.write.parquet(path='/kaggle/working/',
                       mode='append',
                       compression='none')

In [42]:
!rm -f /kaggle/working/*.parquet

In [43]:
yelpData.write.mode('append').option('compression','snappy'). \
    format('parquet').save('/kaggle/working/')

In [ ]:
yelpData.select('review_id').distinct().count()

In [44]:
!rm -f /kaggle/working/*.parquet

In [46]:
yelpData.coalesce(1).write.mode('append').option('compression','none'). \
    option('sep','|').format('csv').save('/kaggle/working/')

KeyboardInterrupt: 

In [ ]:
yelpData. \
    coalesce(1). \
    write. \
    csv(f'/user/{username}/retail_db/order_items',
        sep='|',
        mode='overwrite',
        compression='gzip'
       )

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions', '2')

In [ ]:
spark.sql(f"CREATE DATABASE {username}_demo_db")
spark.sql(f"DROP DATABASE IF EXISTS {username}_demo_db CASCADE")

In [ ]:
spark.catalog.currentDatabase()
spark.catalog.listDatabases()
spark.catalog.setCurrentDatabase(f'{username}_demo_db')

In [ ]:
l = [("X", )]
df = spark.createDataFrame(l, schema="dummy STRING")

In [ ]:
from pyspark.sql.functions import current_date
df.select(current_date()). \
    show()

In [3]:
employees = [
    (1, "Tosk", "Ligk", 1000.0, 
      "united states", "+1 123 456 7890", "123 45 6789"
    ),
     (2, "Kern", "Lork", 1250.0, 
      "India", "+91 234 128 8901", "456 78 9123"
     ),
     (3, "Firk", "John", 750.0, 
      "united KINGDOM", "+84 521 585 1111", "222 33 4444"
     ),
     (4, "Back", "Rown", 1500.0, 
      "AUSTRALIA", "+6657 86 214 558", "789 12 6118"
     )
]

In [ ]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [ ]:
spark.sql('SHOW PARTITIONS orders_part').show()

In [ ]:
df.write.saveAsTable("dual", mode='overwrite')
df.write.insertInto()

In [ ]:
orders. \
    write. \
    saveAsTable(
        'orders_part',
        mode='overwrite',
        partitionBy='order_month'
    )

In [ ]:
spark.catalog. \
    createExternalTable("airport_codes",
                        path=airport_codes_path,
                        source="csv",
                        sep="\t",
                        header="true",
                        inferSchema="true"
                       )

In [ ]:
spark.sql('DESCRIBE FORMATTED airport_codes').show(100, False)

In [ ]:
park.read.table("airport_codes").show()

In [ ]:
spark.catalog.listColumns('airport_codes')

In [ ]:
spark.sql("DROP TABLE dual")

In [2]:
# We can use CASCADE to drop database along with tables.
spark.sql(f"DROP DATABASE IF EXISTS {username}_demo_db CASCADE")
spark.sql("DROP TABLE dual")
spark.sql(f"DROP DATABASE {username}_demo_db")

NameError: name 'spark' is not defined

When we want to create a table using spark.catalog.createTable or using spark.catalog.createExternalTable, we need to specify Schema.

Schema can be inferred or we can pass schema using StructType object while creating the table..

StructType takes list of objects of type StructField.

StructField is built using column name and data type. All the data types are available under pyspark.sql.types.

We need to pass table name and schema for spark.catalog.createTable.

We have to pass path along with name and schema for spark.catalog.createExternalTable

In [ ]:
from pyspark.sql.types import StructField, StructType, \
    IntegerType, StringType, FloatType

In [ ]:
employeesSchema = StructType([
    StructField("employee_id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("salary", FloatType()),
    StructField("nationality", StringType())
])


In [ ]:
employeesSchema.simpleString()

In [ ]:
spark.catalog.createTable("employees", schema=employeesSchema)

In [ ]:
spark.sql('DESCRIBE FORMATTED employees').show(100, truncate=False)

In [ ]:
spark. \
    read. \
    csv(orders_path,
        schema='''order_id INT, order_date DATE,
                  order_customer_id INT, order_status STRING
               '''
       ). \
    withColumn('order_month', date_format('order_date', 'yyyyMM')). \
    write. \
    partitionBy('order_month'). \
    parquet(f'/user/{username}/retail_db/orders_part')

In [ ]:
spark. \
    catalog. \
    createTable('orders_part',
                path=f'/user/{username}/retail_db/orders_part',
                source='parquet'
               )

In [ ]:
spark.sql('SHOW PARTITIONS orders_part').show()

In [ ]:
spark.catalog.recoverPartitions('orders_part')

In [ ]:
spark.read.table('orders_part'). \
    groupBy('order_month'). \
    count(). \
    show()

In [ ]:
airport_codes_df.createTempView("airport_codes_v")